# Cosserat - beam

Srinath Bulusu, Joachim Schöberl

Method from [Sander, 2008]

In [ ]:
import liegroupfem
dir(liegroupfem)

In [ ]:
from ngsolve import *
from ngsolve.meshes import *
from ngsolve.solvers import *
from liegroupfem import *
from ngsolve.webgui import Draw

mesh = Make1DMesh(n=10)

In [ ]:
fes1 = H1(mesh,order=2, dirichlet="left")**3
fes2 = H1(mesh,order=2, dirichlet="left")**4  # for quaternions
fes = fes1*fes2
fes1.ndof, fes2.ndof

Rotation matrix from quaternion q:

def SO3CF(q):
    q = q/Norm(q)
    qv = q[1:4]
    qs = q[0]
    rotmat = qs*OuterProduct(qv, qv) \
        +CF( (0, -qv[2], qv[1],  qv[2], 0, -qv[0],  -qv[1], qv[0], 0)).Reshape((3,3)) \
        -(qv*qv)*Id(3)
    rotmat = 2*rotmat
    rotmat = rotmat + Id(3)
    return rotmat

In [ ]:
phi, q = fes.TrialFunction()

rotation = SO3CF(q)
ngsglobals.msg_level=5
# directional derivative of q -> SO3(q)
gradrotation = rotation.Diff(q, grad(q))  
#print (rotation.Compile(True, keep_files=True))
# print (gradrotation.Compile())

# shear + bending energies:
def W(r, q): 
    shear = grad(r)+CF((1,0,0))-rotation[:,0]
    return 100*InnerProduct(shear,shear) + \
        InnerProduct(gradrotation,gradrotation) + \
        (Norm(q)**2-1)**2

In [ ]:
# shear + bending energies:
def W(r, r_init, q, q_init):
    
    rotmat = SO3CF(q)
    rotmat_init = SO3CF(q_init)
    
    #bend+torsion term
    so3_vec = (rotmat.Diff(q, grad(q)))*rotmat.trans
    so3_init_vec = (rotmat_init.Diff(q_init, grad(q_init)))*rotmat_init.trans
    so3_diff_vec = so3_vec - so3_init_vec
    #TODO: matrix entry wise multiplication with material parameter
    so3_material = 1.
    so3_strain_vec = so3_diff_vec*so3_material
    bend_torsion = Trace(so3_strain_vec * (so3_strain_vec.trans))
    
    #shear+stretch term
    tangent_vec = grad(r)
    tangent_init_vec = grad(r_init)
    #TODO: vector entry wise multiplication with material parameter
    tangent_material = 100.
    #TODO: coupling to rotmat SO3 cross section
    tangent_strain_vec = (rotmat.trans*tangent_vec - rotmat_init.trans*tangent_init_vec)*tangent_material
    shear_stretch = InnerProduct(tangent_strain_vec, tangent_strain_vec)

    #return bend_torsion + shear_stretch
    return bend_torsion + shear_stretch + (Norm(q)**2-1)**2


def get_force_term(r, r_init_gf, q, q_init, force_gf):
    
    rotmat = SO3CF(q)
    rotmat_init = SO3CF(q_init)
    
    return InnerProduct(force_gf, rotmat.trans*grad(r) - rotmat_init.trans*grad(r_init_gf))
    #return InnerProduct(force_gf, r - r_init_gf)

def get_moment_term(q, q_init, moment_gf):
    
    moment_so3_mat = VecCrossMatCF(moment_gf)

    rotmat = SO3CF(q)
    rotmat_init = SO3CF(q_init)
    so3_vec = (rotmat.Diff(q, grad(q)))*Inv(rotmat)
    so3_init_vec = (rotmat_init.Diff(q_init, grad(q_init)))*Inv(rotmat_init)
    so3_diff_vec = so3_vec - so3_init_vec
    #so3_diff_vec = so3_vec
    
    return Trace(so3_diff_vec * (moment_so3_mat.trans))

In [ ]:
a = BilinearForm(fes)
#angle = 0.5*pi
# q_init_cf = (sqrt(1-x*x*cos(0.5*angle)*cos(0.5*angle)), 0, x*sin(0.5*angle), 0)
q_init_cf = (1, 0, 0, 0)
q_init_gf = GridFunction(fes2)
q_init_gf.Set(q_init_cf)

r_init_gf = GridFunction(fes1)
# r_init_gf.Set((sin(x),cos(x),0))
r_init_gf.Set((x,0,0))

a += Variation( W(phi, r_init_gf, q, q_init_gf)* dx )
#a += Variation( W(phi, q)* dx )

#external forces
load = Parameter(1.)
dload = 1.

#boundary forces
#force_dir = (0.,1.,0.)
#force_mod = 1.
#for i in range(0,3):
#    a += Variation( force_mod*force_dir[i]*load*phi[i]*ds("right") )

#external force densities
#force_gf = GridFunction(fes1)
#force_gf.Set((0.,10.*load*x, 0.))
#force_gf.Set((0.,100.*x, 0.))
#force_term = get_force_term(phi, r_init_gf, q, q_init_gf, force_gf)
#a += Variation(force_term*dx)


#external momentum densitites
#moment = (0., 2*load*x, 0.)
moment = (load*x, 0., 0.)
moment_gf = GridFunction(fes1)
moment_gf.Set(moment)
moment_term = get_moment_term(q, q_init_gf, moment_gf)
a += Variation( moment_term*dx )

In [ ]:
print(load.Get())

In [ ]:
gfu = GridFunction(fes)
gfu.components[1].Set( (1,0,0,0) )
#gfu.components[1].Set( q_init_cf )
#gfu.components[0].Set( (0,0,0) )
gfu.components[0].Set( (x,0,0) )
for i in range(5):
    load.Set(load.Get()+dload)
    print("load:", load.Get())
    NewtonMinimization (a, gfu, fes.FreeDofs())

In [ ]:
n = 13  # drawing frames, independent of mesh
pts = [j/n for j in range(n+1)]
uvals = [gfu.components[0](mesh(p)) for p in pts]
q1vals = [ (SO3CF(gfu.components[1])*CF((1,0,0)))(mesh(p)) for p in pts]
q2vals = [ (SO3CF(gfu.components[1])*CF((0,1,0)))(mesh(p)) for p in pts]
q3vals = [ (SO3CF(gfu.components[1])*CF((0,0,1)))(mesh(p)) for p in pts]

# print (uvals)
ll = 0.05
lpts1 = []
for p,u,q1 in zip(pts, uvals, q1vals):
    lpts1 += [p+u[0], u[1], u[2], \
             p+u[0]+ll*q1[0], u[1]+ll*q1[1], u[2]+ll*q1[2]]
lines1 = { "type": "lines", "position": lpts1, "name": "my lines", "color" : "red"}

lpts2 = []
for p,u,q2 in zip(pts, uvals, q2vals):
    lpts2 += [p+u[0], u[1], u[2], \
             p+u[0]+ll*q2[0], u[1]+ll*q2[1], u[2]+ll*q2[2]]
lines2 = { "type": "lines", "position": lpts2, "name": "my lines", "color" : "green"}

lpts3 = []
for p,u,q3 in zip(pts, uvals, q3vals):
    lpts3 += [p+u[0], u[1], u[2], \
             p+u[0]+ll*q3[0], u[1]+ll*q3[1], u[2]+ll*q3[2]]
lines3 = { "type": "lines", "position": lpts3, "name": "my lines", "color" : "blue"}

In [ ]:
Draw (gfu.components[0], deformation=gfu.components[0], objects=[lines1, lines2, lines3]);